# 1. Installs, Imports and Settings

In [25]:
#!pip install spacy==3.1.1 #restart runtime after this
#!python -m spacy download en_core_web_sm

import timer
import csv
import pandas as pd
pd.options.display.max_rows
pd.set_option('display.max_colwidth', None)

import string
import operator
from itertools import islice
from collections import Counter

#!pip install nltk
from nltk import ngrams
import nltk as nltk
from nltk.corpus import stopwords
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('wordnet');

import spacy
nlp = spacy.load("es_core_news_md")

#!pip install pyenchant
import enchant

import time

#  2. Reading files 

In [26]:
df = pd.read_csv("data/alertas_NNAJ_keywords.csv", sep="|")

print("Size of Data:", len(df))
print()
print("Columns : ", list(df.columns))

Size of Data: 1753

Columns :  ['Filename', 'Text', 'Subtype', 'Type', 'Year', 'Path', 'Departamento', 'NNAJ', 'NNAJ_clean_text', 'NNAJ_keywords', 'NNAJ_keywords2', 'NNAJ_keywords3']


# 3. Functions

In [27]:
def get_topKWDs(dfcolumn):

    data = { 
             'keywords': list(Counter(dfcolumn).keys()), 
             'frequency': list(Counter(dfcolumn).values())
           }

    counterdf = pd.DataFrame(data)
    counterdf = counterdf.groupby('keywords').agg({'frequency' : 'sum'}).sort_values('frequency', ascending = False)
    return counterdf

def get_list(colname, n):
    allcols = df[colname].sum()
    if type(allcols) == str:
        allcols = allcols.strip("']['").split("', '")
    return get_topKWDs(allcols)[:n] if n else get_topKWDs(allcols)

In [28]:
list1 = get_list('NNAJ_keywords', None)
list2 = get_list('NNAJ_keywords2', None)
list3 = get_list('NNAJ_keywords3', None)

# 4. Trigrams

In [29]:
list2.head(20)

,frequency
keywords,
grupo armado,2354
armado ilegal,1738
reclutamiento uso,1660
reclutamiento forzado,1335
población civil,1305
uso ilícito,1124
derechos humanos,1067
desplazamiento forzado,1055
uso nnaj,930


In [30]:
list2[list2.frequency>50].describe()

,frequency
count,467.000000
mean,155.659529
std,223.797554
min,51.000000
25%,63.000000
50%,87.000000
75%,135.000000
max,2354.000000


In [31]:
list(list2[:100].index)

['grupo armado',
 'armado ilegal',
 'reclutamiento uso',
 'reclutamiento forzado',
 'población civil',
 'uso ilícito',
 'derechos humanos',
 'desplazamiento forzado',
 'uso nnaj',
 'prevención reclutamiento',
 'ilícito nnaj',
 'bienestar familiar',
 'prevención protección',
 'violencia sexual',
 'alerta temprano',
 'homicidio selectivo',
 'parte grupo',
 'colombiano bienestar',
 'forzado uso',
 'defensoría pueblo',
 'zona rural',
 'mujer nnaj',
 'conflicto armado',
 'nnaj parte',
 'reclutamiento nnaj',
 'actor armado',
 'versión vigente',
 'nnaj mujer',
 'temprano versión',
 'forzado nnaj',
 'menor edad',
 'sexual nnaj',
 'desaparición forzado',
 'vida integridad',
 'mina antipersonal',
 'fuerza público',
 'derecho fundamental',
 'riesgo reclutamiento',
 'violación derechos',
 'nombre prevención',
 'comisión prevención',
 'nnaj año',
 'situación riesgo',
 'explotación sexual',
 'restricción movilidad',
 'interposición población',
 'selectivo múltiple',
 'artefacto explosivo',
 'nnaj gr

In [32]:
get_list('NNAJ_keywords', 50)

,frequency
keywords,
nnaj,11820
armado,4447
reclutamiento,4409
grupo,4040
uso,3520
población,3476
forzado,3260
riesgo,3167
municipio,2767


## 5. Exporting to html

In [34]:
!jupyter nbconvert --to html 9_NNAJ_Report.ipynb

[NbConvertApp] Converting notebook 9_NNAJ_Report.ipynb to html
[NbConvertApp] Writing 603469 bytes to 9_NNAJ_Report.html
